In [ ]:
Project: Data Minining Project for X company

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

df = pd.read_csv("./clean_processed_tweet_data.csv")


In [2]:
df = pd.read_csv("./clean_processed_tweet_data.csv")

In [3]:
df.head()

,Unnamed: 0,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,0,2021-06-18 17:55:49+00:00,"<a href=""http://twitter.com/download/iphone"" r...","🚨Africa is ""in the midst of a full-blown third...",0.166667,0.188889,en,548.0,612.0,ketuesriche,551,351,NaN,[],NaN,Mass
1,1,2021-06-18 17:55:59+00:00,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Dr Moeti is head of WHO in Africa, and one of ...",0.133333,0.455556,en,195.0,92.0,Grid1949,66,92,NaN,[],NaN,"Edinburgh, Scotland"
2,2,2021-06-18 17:56:07+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Thank you @research2note for creating this ama...,0.316667,0.483333,en,2.0,1.0,LeeTomlinson8,1195,1176,NaN,"[{'text': 'red4research', 'indices': [87, 100]}]",NaN,NaN
3,3,2021-06-18 17:56:10+00:00,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1580.0,899.0,RIPNY08,2666,2704,NaN,"[{'text': 'Covid19', 'indices': [145, 153]}]",NaN,NaN
4,4,2021-06-18 17:56:20+00:00,"<a href=""http://twitter.com/download/android"" ...",I think it’s important that we don’t sell COVA...,0.280000,0.620000,en,72.0,20.0,pash22,28250,30819,NaN,[],NaN,United Kingdom


In [4]:
Clean_Tweet = df[['original_text','polarity']]

In [5]:
def text_category (polarity):
    if polarity > 0:
        return 'positive'
    if polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [6]:
score = pd.Series([text_category(row_value) for row_value in Clean_Tweet['polarity']])

In [7]:
Clean_Tweet = pd.concat([Clean_Tweet, score.rename("score")], axis=1)

In [8]:
Clean_Tweet.head()

,original_text,polarity,score
0,"🚨Africa is ""in the midst of a full-blown third...",0.166667,positive
1,"Dr Moeti is head of WHO in Africa, and one of ...",0.133333,positive
2,Thank you @research2note for creating this ama...,0.316667,positive
3,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,positive
4,I think it’s important that we don’t sell COVA...,0.280000,positive


In [9]:
len(Clean_Tweet['polarity'])

6092

In [16]:
scoremap = pd.Series([1 if row_value == 'positive' else 0 for row_value in Clean_Tweet['score']])

In [17]:

len(scoremap)

6092

In [20]:
Clean_Tweet['scoremap'] = scoremap

In [21]:
X = Clean_Tweet['original_text']
y = Clean_Tweet['scoremap']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [23]:
print(X_train.size)
print(X_test.size)
print(y_train.size)
print(y_test.size)

4569
1523
4569
1523


In [24]:
Clean_Tweet.head()

,original_text,polarity,score,scoremap
0,"🚨Africa is ""in the midst of a full-blown third...",0.166667,positive,1
1,"Dr Moeti is head of WHO in Africa, and one of ...",0.133333,positive,1
2,Thank you @research2note for creating this ama...,0.316667,positive,1
3,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,positive,1
4,I think it’s important that we don’t sell COVA...,0.280000,positive,1


In [25]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
# scaling the input
clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=9000, tol=1e-2))

In [28]:
count_vect = CountVectorizer(3,3)
X_train = X_train.replace(np.nan, '', regex=True)
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts = X_train_counts.toarray()

In [29]:
clf.fit(X_train_counts, y_train)

/home/natty/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sgdclassifier', SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=9000, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=0.01, verbose=0, warm_start=False))])

In [30]:
X_test = X_test.replace(np.nan, '', regex=True)
# use transform not fit_transform
X_test_counts = count_vect.transform(X_test)
X_test_counts = X_test_counts.toarray()
# prediction = clf.prevaluedict(X_test_counts)
prediction = clf.predict(X_test_counts)

/home/natty/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
# making prediction
prediction = clf.predict(X_test_counts)

/home/natty/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [32]:
np.mean(prediction == y_test)

0.9770190413657256